# **1.Import Pytorch**

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models.feature_extraction as feature_extraction
from torchsummary import summary

import os
import copy

no_cuda = False
use_gpu = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")

# **2.Load Fashion MNIST Dataset**

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

batch_size = 32

#Dataset
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

#Dataloader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# **3.Create and train a NN model**

In [3]:
class ToyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.nn1 = nn.Linear(28*28, 120)
    self.nn2 = nn.Linear(120, 84)
    self.nn3 = nn.Linear(84, 10)

  def forward(self, x):
    x = x.view(-1, 28 * 28) #transform 28*28 figure to 784 vector
    x = F.relu(self.nn1(x))
    x = F.relu(self.nn2(x))
    x = self.nn3(x)
    return x

In [4]:
FP32_model = ToyModel().to(device)
summary(FP32_model,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 120]          94,200
            Linear-2                   [-1, 84]          10,164
            Linear-3                   [-1, 10]             850
Total params: 105,214
Trainable params: 105,214
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.40
Estimated Total Size (MB): 0.41
----------------------------------------------------------------


In [5]:
#train model
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  #Set the model to train mode
  model.train()
  for batch, (x, y) in enumerate(dataloader):
    if use_gpu:
      x, y = x.cuda(), y.cuda()
    optimizer.zero_grad()
    #forward
    pred = model(x)

    #loss
    loss = loss_fn(pred, y)

    #backward
    loss.backward()

    #optimize
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(x)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  #set model to evaluate mode
  model.eval()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for x, y in dataloader:
      if use_gpu:
        x, y = x.cuda(), y.cuda()
      pred = model(x)
      test_loss = loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item() #calculate accuracy
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
learning_rate = 1e-3
epochs = 3
loss_fn = nn.CrossEntropyLoss() #define loss function
optimizer = torch.optim.SGD(FP32_model.parameters(), lr=learning_rate, momentum=0.9)  #define optimizer

FP32_model.to(device) #let model on GPU

ToyModel(
  (nn1): Linear(in_features=784, out_features=120, bias=True)
  (nn2): Linear(in_features=120, out_features=84, bias=True)
  (nn3): Linear(in_features=84, out_features=10, bias=True)
)

In [7]:
for i in range(epochs):
  print(f"Epoch {i+1}\n-------------------------------")
  train_loop(train_loader, FP32_model, loss_fn, optimizer)
  test_loop(test_loader, FP32_model, loss_fn)

Epoch 1
-------------------------------
loss: 2.321159  [   32/60000]


loss: 1.927126  [ 3232/60000]
loss: 1.303721  [ 6432/60000]
loss: 1.097808  [ 9632/60000]
loss: 0.884623  [12832/60000]
loss: 0.715995  [16032/60000]
loss: 0.711115  [19232/60000]
loss: 0.685886  [22432/60000]
loss: 0.459083  [25632/60000]
loss: 0.757259  [28832/60000]
loss: 0.643529  [32032/60000]
loss: 0.396894  [35232/60000]
loss: 0.497167  [38432/60000]
loss: 0.473881  [41632/60000]
loss: 0.502233  [44832/60000]
loss: 0.758501  [48032/60000]
loss: 0.417074  [51232/60000]
loss: 0.376126  [54432/60000]
loss: 0.539204  [57632/60000]
Test Error: 
 Accuracy: 80.0%, Avg loss: 0.000896 

Epoch 2
-------------------------------
loss: 0.392918  [   32/60000]
loss: 0.654713  [ 3232/60000]
loss: 0.483461  [ 6432/60000]
loss: 0.316461  [ 9632/60000]
loss: 0.710897  [12832/60000]
loss: 0.409961  [16032/60000]
loss: 0.341131  [19232/60000]
loss: 0.660500  [22432/60000]
loss: 0.561974  [25632/60000]
loss: 0.577379  [28832/60000]
loss: 0.375154  [32032/60000]
loss: 0.479795  [35232/60000]
loss: 0.

# **4. Quantize**

下面兩格為需要實作的程式碼，請完成normal、clip的scale及zero point算法並且根據計算出來的s, z去進行tensor的quantize

算法可以參考EAI Lab6 Page5、6的部分或是example的程式碼

normal: 15%, clip: 15%, 結報請截圖實作部分的程式碼以及最後的兩種model的Accuracy

In [8]:
def Calculate_scale_zero_point(x, mode="normal"):
  if mode == "normal":
    '''
    請完成以下程式碼
    '''
    q_min, q_max = -128, 127  # int8
    min_val, max_val = np.min(x.detach().numpy()), np.max(x.detach().numpy()) # get min/max value of x

    scale = (max_val - min_val) / (q_max - q_min) # calculate scale
    zero_point = round(q_min - min_val / scale) # calculate zero_point

  elif mode == "clip":
    '''
    請完成以下程式碼
    '''
    q_min, q_max = -256, 255
    min_val, max_val = np.min(x.detach().numpy()), np.max(x.detach().numpy()) # get min/max value of x

    scale = (max_val - min_val) / (q_max - q_min) # calculate scale
    zero_point = round(q_min - min_val / scale)

  return scale, zero_point

In [9]:
class Quantize_per_tensor(nn.Module):
  def __init__(self, x, scale, zero_point, mode="normal"):
    super().__init__()
    self.tensor = x
    self.scale = scale
    self.zero_point = zero_point
    self._quantize(mode)

  def repr(self):
    return self.qtensor

  def int_repr(self):
    return self.qtensor_int

  def _get_scale_zero(self):
    return self.scale, self.zero_point

  def _quantize(self, mode):
    if mode == "normal":
      self.qtensor_int = torch.round(self.tensor / self.scale + self.zero_point)  # q = round(r/s + zp)
      self.qtensor = (self.qtensor_int - self.zero_point) * self.scale  # rq = (q - zp) * scale

    elif mode == "clip":
      self.qtensor_int = torch.round(self.tensor / self.scale + self.zero_point)
      self.qtensor_int = torch.clamp(self.qtensor_int, -128, 127) # clamp to [-128, 127]
      self.qtensor = (self.qtensor_int - self.zero_point) * self.scale

In [10]:
class QuantizedLinear(nn.Module):
  def __init__(self, in_features, out_features, weight, bias, scale, zero_point, mode):
    super(QuantizedLinear, self).__init__()
    self.in_features = in_features
    self.out_features = out_features
    self.mode = mode
    self.scale, self.zero_point = scale, zero_point
    self.weight = self._weight_quantize(weight)
    self.bias = bias


  def forward(self, x):
    x = torch.matmul(x, self.weight.t())
    output = Quantize_per_tensor(x, self.scale, self.zero_point, mode=self.mode).repr() + self.bias

    return output

  def _weight_quantize(self, weight):
    s, z = Calculate_scale_zero_point(weight)
    qweight = Quantize_per_tensor(weight, s, z, mode=self.mode)
    return qweight.repr()

  def __repr__(self):
    return f'QuantizedLinear(in_features={self.in_features}, out_features={self.out_features}, scale={self.scale}, zero_point={self.zero_point})'

class QuantizedLinearReLU(nn.Module):
  def __init__(self, in_features, out_features, weight, bias, scale, zero_point, mode):
    super(QuantizedLinearReLU, self).__init__()
    self.in_features = in_features
    self.out_features = out_features
    self.mode = mode
    self.scale, self.zero_point = scale, zero_point
    self.weight = self._weight_quantize(weight)
    self.bias = bias


  def forward(self, x):
    x = torch.matmul(x, self.weight.t())
    output = Quantize_per_tensor(x, self.scale, self.zero_point, mode=self.mode).repr() + self.bias
    output = F.relu(output)
    return output

  def _weight_quantize(self, weight):
    s, z = Calculate_scale_zero_point(weight)
    qweight = Quantize_per_tensor(weight, s, z, mode=self.mode)
    return qweight.repr()

  def __repr__(self):
    return f'QuantizedLinearReLU(in_features={self.in_features}, out_features={self.out_features}, scale={self.scale}, zero_point={self.zero_point})'

In [11]:
class QuantizedModel(nn.Module):
  def __init__(self, model, scale, zero_point, mode="normal"):
    super(QuantizedModel, self).__init__()
    self.weight_dic = []
    self.bias_dic = []
    self.scale, self.zero_point = scale, zero_point #Scale and zero point of all layer
    self.mode = mode

    self._get_weight()
    self.nn1 = QuantizedLinearReLU(in_features=28*28, out_features=120, weight=self.weight_dic[0], bias=self.bias_dic[0], scale=self.scale[1], zero_point=self.zero_point[1], mode=self.mode)
    self.nn2 = QuantizedLinearReLU(in_features=120, out_features=84, weight=self.weight_dic[1], bias=self.bias_dic[1], scale=self.scale[2], zero_point=self.zero_point[2], mode=self.mode)
    self.nn3 = QuantizedLinear(in_features=84, out_features=10, weight=self.weight_dic[2], bias=self.bias_dic[2], scale=self.scale[3], zero_point=self.zero_point[3], mode=self.mode)

  def forward(self, x):
    x = x.view(-1, 28 * 28)
    x = Quantize_per_tensor(x, self.scale[0], self.zero_point[0], mode=self.mode).repr()
    x = self.nn1(x)
    x = self.nn2(x)
    x = self.nn3(x)
    x = x.dequantize()
    return x

  def _get_weight(self):
    for name, paras in model.named_parameters():
      if "weight" in name:
        self.weight_dic.append(paras)
      elif "bias" in name:
        self.bias_dic.append(paras)

# Normal quantization

In [12]:
model = copy.deepcopy(FP32_model)

In [13]:
scale_dic = []
zero_dic = []

#Calibrate to compute s、z of all layer at the same time
for batch in train_loader:
  input, label = batch
  for node in ['x', 'relu', 'relu_1', 'nn3']:
    extractor = feature_extraction.create_feature_extractor(model, [node]).cpu()
    output = extractor(input)[node]
    q_min, q_max = -128, 127
    min_val, max_val = np.min(output.detach().numpy()), np.max(output.detach().numpy())
    scale = (max_val - min_val) / (q_max - q_min)
    zero = round(q_min - min_val / scale)
    q = Quantize_per_tensor(output, scale=scale, zero_point=zero, mode="normal")
    scale_dic.append(scale)
    zero_dic.append(zero)
  break

print(scale_dic)
print(zero_dic)


[0.00784313725490196, 0.02852339277080461, 0.03299593457988664, 0.0984071320178462]
[0, -128, -128, -18]


/home/nckusoc/miniconda3/envs/eai-lab/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/nckusoc/miniconda3/envs/eai-lab/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/nckusoc/miniconda3/envs/eai-lab/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/nckusoc/miniconda3/envs/eai-lab/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


In [14]:
Quantized_normal_model = QuantizedModel(model, scale_dic, zero_dic, mode="normal")
print(Quantized_normal_model)

QuantizedModel(
  (nn1): QuantizedLinearReLU(in_features=784, out_features=120, scale=0.02852339277080461, zero_point=-128)
  (nn2): QuantizedLinearReLU(in_features=120, out_features=84, scale=0.03299593457988664, zero_point=-128)
  (nn3): QuantizedLinear(in_features=84, out_features=10, scale=0.0984071320178462, zero_point=-18)
)


# Clip quantization

In [15]:
model = copy.deepcopy(FP32_model)

In [16]:
scale_dic = []
zero_dic = []

#Calibrate to compute s、z of all layer at the same time
for batch in train_loader:
  input, label = batch
  for node in ['x', 'relu', 'relu_1', 'nn3']:
    extractor = feature_extraction.create_feature_extractor(model, [node]).cpu()
    output = extractor(input)[node]
    q_min, q_max = -128, 127
    min_val, max_val = np.min(output.detach().numpy()), np.max(output.detach().numpy())
    scale = (max_val - min_val) / (q_max - q_min)
    zero = round(q_min - min_val / scale)
    q = Quantize_per_tensor(output, scale=scale, zero_point=zero, mode="clip")
    scale_dic.append(scale)
    zero_dic.append(zero)
  break

print(scale_dic)
print(zero_dic)


[0.00784313725490196, 0.02732350966509651, 0.03290563471177045, 0.0984171848671109]
[0, -128, -128, -16]


In [17]:
Quantized_clip_model = QuantizedModel(model, scale_dic, zero_dic, mode="clip")
print(Quantized_clip_model)

QuantizedModel(
  (nn1): QuantizedLinearReLU(in_features=784, out_features=120, scale=0.02732350966509651, zero_point=-128)
  (nn2): QuantizedLinearReLU(in_features=120, out_features=84, scale=0.03290563471177045, zero_point=-128)
  (nn3): QuantizedLinear(in_features=84, out_features=10, scale=0.0984171848671109, zero_point=-16)
)


In [18]:
#define evaluate function
def Evaluate(model, loader):
  total = 0
  correct = 0
  with torch.no_grad():
    for data in loader:
      images, labels = data
      outputs = model(images)
      # the class with the highest energy is what we choose as prediction
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  test_loss = 0

  print("========================================= PERFORMANCE =============================================")
  print('\nAccuracy: {}/{} ({:.0f}%)\n'.format( correct, total,100. * correct / total))

In [19]:
#Normal quantize
Evaluate(Quantized_normal_model, test_loader)

========================================= PERFORMANCE =============================================

Accuracy: 8442/10000 (84%)



In [20]:
#Clip quantize
Evaluate(Quantized_clip_model, test_loader)

========================================= PERFORMANCE =============================================

Accuracy: 8427/10000 (84%)

